In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#import finnhub
import cufflinks as cf
cf.go_offline
from datetime import datetime
from tqdm import trange
from pandas_datareader import data
import yfinance as yf

import warnings
warnings.filterwarnings('ignore')

In C:\Users\Emeka\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Emeka\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Emeka\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Emeka\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Emeka\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

# Load Data

In [2]:
#start = '2011-2-1'
#data = yf.download('USDT_BTC', start)
df = pd.read_csv('TSLA - Feb 11 to 21.csv', index_col='Date', parse_dates=True)
#df.set_index('Date', inplace=True)
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2011-02-01,4.862000,4.946000,4.708000,4.782000,4.782000,3539000
2011-02-02,4.832000,4.836000,4.734000,4.788000,4.788000,2847500
2011-02-03,4.764000,4.780000,4.630000,4.726000,4.726000,2560000
2011-02-04,4.688000,4.734000,4.644000,4.692000,4.692000,2720000
2011-02-07,4.652000,4.652000,4.576000,4.614000,4.614000,4475500
...,...,...,...,...,...,...
2021-01-25,855.000000,900.400024,838.820007,880.799988,880.799988,41173400
2021-01-26,891.380005,895.900024,871.599976,883.090027,883.090027,23131600
2021-01-27,870.349976,891.500000,858.659973,864.159973,864.159973,27334000


# Define fuctions for chart and strategy

In [3]:
#Define function for interactive chart
def chart_df(df):
    """
    Plot interactive charts
    
    Args:
        df(DataFrame): OPHLCAV Dataframe
        
        Returns:
            Plot: OPHLCAV Price chart
    """
    qt = cf.QuantFig(df, kind='candle', title='TESLA')
    qt.add_volume()
    qt.add_ema([50, 200], width=2, color=['green', 'red'], legendgroup=True)
    plot = qt.iplot(asFigure=True)
    return plot

In [4]:
#Backtest, strategy and optimize
class Backtest:
    """Defines, optimize and backtest trading strategy"""
    
    def __init__(self, df, capital, order_size):
        """Iniitialize our backtest
        
            Args:
                df(DataFrame): OPHLCAV Dataframe
                capital: Amount to be spent on shares
                order_size: Number of shares to buy on each buy and sale order
        """
        global amount
        amount = capital
        self.df = df.copy()
        self.amount = capital
        self.order_size = order_size
        
    def strategy(self, short_ma, long_ma):
        """Backtest the performance of the moving average crossover
        
            Args:
                short_ma: Short moving average window
                long_ma: Long moving average window
            
            Returns:
                Dataframe backtest result
                
        """
        
        b_df = self.df[['Close']]
        b_df.columns = ['Price']
        b_df['Short MA'] = b_df['Price'].rolling(short_ma).mean()
        b_df['Long MA'] = b_df['Price'].rolling(long_ma).mean()
        b_df['Diff'] = b_df['Short MA'] - b_df['Long MA']
        b_df['Signal'] = b_df['Diff'].apply(lambda x: (x*0) + 1 if x > 0 else (x*0) - 1 if x < 0 else 0)
        b_df['Long'] = b_df['Price'] * (b_df['Signal'][b_df['Signal'] == 1]) * self.order_size
        b_df['Short'] = b_df['Price'] * (b_df['Signal'][b_df['Signal'] == -1]) * self.order_size
        b_df['Returns'] = b_df['Long'].pct_change().fillna(0) + b_df['Short'].pct_change().fillna(0)
        b_df['Cum Return'] = (b_df['Returns']+1).cumprod()
        b_df['Portfolio'] = b_df['Cum Return'] * self.amount
        return b_df
    
    def optimize(self, long_mas, short_mas):
        """Select the best parameter for the backtest of trading strategy
            
            Args:
                short_mas(int): Maximum short moving average window
                long_mas(int): maximum long moving average window
            Returns:
                Dataframe: optimum strategy parameters
                
        """
        parameters = []
        for long_ma in trange(0, long_mas+1, 5):
            for short_ma in range(0, short_mas+1, 5):
                if long_ma > short_ma:
                    b_df = Backtest.strategy(self, short_ma, long_ma)
                    port_return = round(((b_df['Portfolio'].iloc[-1]/b_df['Portfolio'].iloc[0])) * 100,2)
                    try:
                        sharpe_ratio = round((b_df['Returns'].mean()/b_df['Returns'].std()), 2)
                    except:
                        sharpe_ratio = None
                    parameter = {'Short MA': short_ma, 'Long MA': long_ma, 'Total Portfolio Return': port_return,
                                         'Sharpe Ratio': sharpe_ratio}
                    parameters.append(parameter)
        parameters_df = pd.DataFrame(parameters).set_index(['Short MA', 'Long MA'])
        best_parameters = parameters_df[parameters_df['Total Portfolio Return'] == parameters_df['Total Portfolio Return'].max()]
        self.short_ma = best_parameters.index[0][0]
        self.long_ma = best_parameters.index[0][1]
        return best_parameters
    
    #plotting the backtest result
    def plot(self):
        """Plots the backtest result"""
        
        b_df = Backtest.strategy(self, self.short_ma, self.long_ma)
        b_df[['Price', 'Short MA', 'Long MA']].plot(title='Price', figsize=(12,8))
        plt.show()
        b_df['Signal'].plot(title='Signals', figsize=(12,8))
        plt.show()
        b_df['Returns'].plot(title='Returns', figsize=(12,8))
        plt.show()
        b_df['Cum Return'].plot(title='Cumulative Returns', figsize=(12,8))
        plt.show()
        return b_df
    

In [5]:
bt = Backtest(df, 1000, 1)

In [6]:
bt.optimize(50,200)

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  3.65it/s]


,,Total Portfolio Return,Sharpe Ratio
Short MA,Long MA,,
10,15,3154943.98,0.08


# EDA & Feature Enginerring

# Define our model

In [ ]:
#df.describe().T

In [ ]:
#df.isnull().sum()

In [ ]:
#df['previous_day_admission_influenza_confirmed_7_day_sum']#.isnull().sum()

**Dealing with missing values**

In [ ]:
#missing_val = ['N/a', 0, '--', ' ']

In [ ]:
#df = pd.read_csv(url, na_values=missing_val)

In [ ]:
#df.isnull().sum().sum()

In [ ]:
#df.isnull().values.any()

In [ ]:
#df.head()

In [ ]:
#df['collection_week'] = pd.to_datetime(df['collection_week'])

In [ ]:
#df['collection_week'][0]

In [ ]:
#Drop some irrelevant columns
#df = df.drop('hospital_pk', axis=1)

In [ ]:
#df.columns

In [ ]:
#total_missing = df.isnull().sum().sort_values(ascending=False)
#perc_missing = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
#missing = pd.concat([total_missing, perc_missing], axis=1, keys=['Total', 'Percentage'])
#missing.head(50)

In [ ]:
#df.isnull().sum().count()